In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese_gaoq1/main')

In [0]:
%tensorflow_version 1.x

In [3]:
import tensorflow as tf
import time

from pathlib import Path

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0
GPU Enabled: False


In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

def parse_fn(text):
  parse = lambda text: [[params['char2idx'].get(c, len(params['char2idx'])) for c in list(text)]]
  return {'words': parse(text)}

In [5]:
params = {'export_dir': '../model/lstm_seq2seq_export'}

params['char2idx'] = get_vocab('../vocab/char.txt')

params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

subdirs = [x for x in Path(params['export_dir']).iterdir()
           if x.is_dir() and 'temp' not in str(x)]
latest = str(sorted(subdirs)[-1])
print(latest)

predict_fn = tf.contrib.predictor.from_saved_model(latest)

../model/lstm_seq2seq_export/1577431850
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../model/lstm_seq2seq_export/1577431850/variables/variables


In [0]:
while True:
  text_inp = input('Input:')
  t0 = time.time()
  predictions = predict_fn(parse_fn(text_inp))
  #print('Shape:', predictions['output'].shape)
  for j in range(3):
    indices = [params['idx2char'].get(idx, len(params['idx2char'])) for idx in predictions['output'][0, :, j] if (idx != 0 and idx != 2)]
    sent = ' '.join(indices)
    print('Rank_{}:'.format(j+1), sent)
  print('Cost %.2f sec' % (time.time() - t0))
  print()

Input:你好
Rank_1: 你 好
Rank_2: 你 好 你 好
Rank_3: 你 好 ， 你 好
Cost 0.66 sec

Input:早上好
Rank_1: 早 上 好
Rank_2: 早 上 好 ！
Rank_3: 早 安
Cost 0.11 sec

Input:晚上好
Rank_1: 晚 上 好
Rank_2: 晚 上 好 ！
Rank_3: 晚 安
Cost 0.10 sec

Input:再见
Rank_1: 再 见
Rank_2: 哈 哈 哈
Rank_3: 哈 哈
Cost 0.11 sec

Input:好久不见
Rank_1: 好 久 不 见
Rank_2: 好 久 不 见 啊
Rank_3: 好 久 不 见 。
Cost 0.10 sec

Input:想死你了
Rank_1: 我 也 想 你
Rank_2: 我 也 是
Rank_3: 哈 哈 哈 哈 哈 哈 哈 哈 哈 哈
Cost 0.10 sec

Input:谢谢你
Rank_1: 不 客 气
Rank_2: 不 用 谢
Rank_3: 客 气
Cost 0.09 sec

Input:爱你
Rank_1: 么 么 哒
Rank_2: 我 也 爱 你
Rank_3: 爱 你
Cost 0.10 sec

Input:你叫什么名字
Rank_1: 名 字
Rank_2: 叫 什 么 名 字
Rank_3: 名 字 叫 什 么 名 字
Cost 0.10 sec

Input:你几岁了
Rank_1: 十 岁
Rank_2: 岁 月
Rank_3: 一 岁
Cost 0.04 sec

Input:现在几点了
Rank_1: 一 点 点
Rank_2: 八 点 多
Rank_3: 八 点 半
Cost 0.10 sec

Input:今天天气怎么样
Rank_1: 今 天 天 气 好
Rank_2: 今 天 天 气 不 错
Rank_3: 今 天 天 气 好 ， 今 天 天 气
Cost 0.12 sec

Input:我们现在在哪里
Rank_1: 你 在 哪 里
Rank_2: 你 在 哪 里 ？
Rank_3: 你 在 哪 里 啊
Cost 0.07 sec

Input:你能给我讲个笑话吗
Rank_1: 不 能
Rank_2: 哈 哈 哈 哈 哈 哈 哈 哈 哈 